# Installation

Note: If you are on Binder, you don't need to execute the following command.

In [ ]:
!pip install nexusforge

# Configuration

This notebook presents a set of configuation options to set up when creating a knowledge graph forge session. Refer to the [Nexus Forge docs](https://nexus-forge.readthedocs.io/en/latest/interaction.html#forge) to learn more about all the possible configuration options.

In [1]:
config = dict()

## Configure for Demo
This configuration is for testing Nexus Forge features without using or deploying a persistent store. Not all features are  accessible with the demo configuration. The demo configuration is therefore not recommendeded for production use. 

### Model

In [2]:
config['Model'] = {
    "name": "DemoModel",
    "origin": "directory",
    "source": "../../../tests/data/demo-model/",
}

### Store

In [3]:
config["Store"] = {
    "name": "DemoStore",
    "versioned_id_template": "{x.id}?_version={x._store_metadata.version}"
}

### Resolvers

#### sourced from a directory

In [4]:
config["Resolvers"] = {
    "terms": [
        {
            "resolver": "DemoResolver",
            "origin": "directory",
            "source": "../../../tests/data/demo-resolver/",
            "targets": [
                {
                    "identifier": "sex",
                    "bucket": "sex.json"
                }
            ],
            "result_resource_mapping": "../../configurations/demo-resolver/term-to-resource-mapping.hjson"
        }
    ],
    "entities": [
        {
            "resolver": "DemoResolver",
            "origin": "directory",
            "source": "../../../tests/data/demo-resolver/",
            "targets": [
                {
                    "identifier": "agents",
                    "bucket": "agents.json"
                }
            ],
            "result_resource_mapping": "../../configurations/demo-resolver/entity-to-resource-mapping.hjson"
        }
    ]
}

## Configure Nexus Forge to use with [Blue Brain Nexus Delta](https://bluebrainnexus.io/docs/delta/api/current/index.html) as a Store

### Create a project to work with in the BlueBrainNexus sandbox

#### Get a token

The [Nexus sandbox application](https://sandbox.bluebrainnexus.io/web) can be used to login and get a token.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/copy-token.png)


In [ ]:
import getpass

token = getpass.getpass()

#### Set a project name

In [ ]:
project ="myProject"

#### Create the project
The next cell will fail if the project alreaaday exist

In [ ]:
import nexussdk as nexus

org ="dke"
description = "My project description"
endpoint = "https://staging.nexus.ocp.bbp.epfl.ch/v1"

nexus.config.set_environment(endpoint)
nexus.config.set_token(token)

nexus.projects.create(org_label=org,project_label=project, description=description)

### Model

#### RDFModel

This model supports the W3C SHACL schema language. Let use examples of SHACL schemas from https://github.com/INCF/neuroshapes. SHACL schemas can be loaded either from a directory or from a store. 

##### sourced from a directory

In [ ]:
neuroshapes_path = "../../models/neuroshapes"

In [ ]:
! rm -Rf $neuroshapes_path

In [ ]:
! git clone https://github.com/INCF/neuroshapes.git $neuroshapes_path

In [ ]:
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/dataset $neuroshapes_path/shapes/neurosciencegraph/commons/
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/activity $neuroshapes_path/shapes/neurosciencegraph/commons/
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/entity $neuroshapes_path/shapes/neurosciencegraph/commons/
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/ontology $neuroshapes_path/shapes/neurosciencegraph/commons/
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/person $neuroshapes_path/shapes/neurosciencegraph/commons/
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/contribution $neuroshapes_path/shapes/neurosciencegraph/commons/

In [ ]:
config['Model'] = {
    "name": "RdfModel",
    "origin": "directory",
    "source": f"{neuroshapes_path}/shapes/neurosciencegraph/commons/",
    "context": {
        "iri": "../../models/neuroshapes_context.json"
    },
}

##### sourced from BlueBrainNexus store

In [ ]:
shacl_schema_bucket = "neurosciencegraph/datamodels"

In [ ]:
config['Model'] = {
    "name": "RdfModel",
    "origin": "store",
    "source": "BlueBrainNexus",
    "context": {
        "iri": "https://bbp.neuroshapes.org",
        "bucket": shacl_schema_bucket
    }
}

### Store

In [ ]:
config["Store"] = {
    "name": "BlueBrainNexus",
    "endpoint": endpoint,
    "searchendpoints":{
        "sparql":{
            "endpoint":"https://bluebrain.github.io/nexus/vocabulary/defaultSparqlIndex"
        }
    },
    "bucket": f"{org}/{project}",
    "token": token,
    "versioned_id_template": "{x.id}?rev={x._store_metadata._rev}",
    "file_resource_mapping": "../../configurations/nexus-store/file-to-resource-mapping.hjson"
}

### Resolvers

#### sourced from a store

In [ ]:
ontology_bucket = "neurosciencegraph/datamodels"

In [ ]:
config["Resolvers"] = {
    "ontology": [
        {
            "resolver": "OntologyResolver",
            "origin": "store",
            "source": "BlueBrainNexus",
            "targets": [
                {
                    "identifier": "terms",
                    "bucket": ontology_bucket
                }
            ],
            "result_resource_mapping": "../../configurations/nexus-resolver/term-to-resource-mapping.hjson"
        }
    ]
}

#### sourced from a directory

In [ ]:
config["Resolvers"] = {
    "terms": [
        {
            "resolver": "DemoResolver",
            "origin": "directory",
            "source": "../../../tests/data/demo-resolver/",
            "targets": [
                {
                    "identifier": "sex",
                    "bucket": "sex.json"
                }
            ],
            "result_resource_mapping": "../../configurations/demo-resolver/term-to-resource-mapping.hjson"
        }
    ],
    "entities": [
        {
            "resolver": "DemoResolver",
            "origin": "directory",
            "source": "../../../tests/data/demo-resolver/",
            "targets": [
                {
                    "identifier": "agents",
                    "bucket": "agents.json"
                }
            ],
            "result_resource_mapping": "../../configurations/demo-resolver/entity-to-resource-mapping.hjson"
        }
    ]
}

## Configure formatters

In [5]:
config["Formatters"] = {
    "identifier": "https://kg.example.ch/{}/{}",
}

## Save configuration

In [6]:
import yaml

In [7]:
with open("../../configurations/forge.yml", "w") as f:
    yaml.dump(config, f)

You are all set up!